In [1]:
import cv2
import numpy as np

In [2]:
def get_illumination_channel(I, w):
    M, N, _ = I.shape
    padded = np.pad(I, ((int(w/2), int(w/2)), (int(w/2), int(w/2)), (0, 0)), 'edge')
    darkch = np.zeros((M, N))
    brightch = np.zeros((M, N))
    
    for i, j in np.ndindex(darkch.shape):
        darkch[i, j]  =  np.min(padded[i:i + w, j:j + w, :])
        brightch[i, j] = np.max(padded[i:i + w, j:j + w, :])
    
    return darkch, brightch

In [22]:
I = cv2.imread('./darkInCars.jpeg')
darkch,brightch = get_illumination_channel(I, 50)
print(darkch,brightch)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [[8. 8. 8. ... 8. 8. 8.]
 [8. 8. 8. ... 8. 8. 8.]
 [8. 8. 8. ... 8. 8. 8.]
 ...
 [5. 5. 5. ... 4. 4. 4.]
 [5. 5. 5. ... 4. 4. 4.]
 [5. 5. 5. ... 4. 4. 4.]]


In [10]:
def get_atmosphere(I, brightch, p=0.1):
    M, N = brightch.shape
    flatI = I.reshape(M*N, 3) # reshaping image array
    flatbright = brightch.ravel() #flattening image array

    searchidx = (-flatbright).argsort()[:int(M*N*p)] # sorting and slicing
    A = np.mean(flatI.take(searchidx, axis=0), dtype=np.float64, axis=0)
    return A

In [23]:
A = get_atmosphere(I, brightch)

In [24]:
def get_initial_transmission(A, brightch):
    A_c = np.max(A)
    init_t = (brightch-A_c)/(1.-A_c) # finding initial transmission map
    return (init_t - np.min(init_t))/(np.max(init_t) - np.min(init_t)) # normalized initial transmission map


In [26]:
init_t = get_initial_transmission(A, brightch)
cv2.imwrite('./test.jpg', init_t)

True

In [19]:
def get_corrected_transmission(I, A, darkch, brightch, init_t, alpha, omega, w):
    im = np.empty(I.shape, I.dtype);
    for ind in range(0, 3):
        im[:, :, ind] = I[:, :, ind] / A[ind] #divide pixel values by atmospheric light
    dark_c, _ = get_illumination_channel(im, w) # dark channel transmission map
    dark_t = 1 - omega*dark_c # corrected dark transmission map
    corrected_t = init_t # initializing corrected transmission map with initial transmission map
    diffch = brightch - darkch # difference between transmission maps
 
    for i in range(diffch.shape[0]):
        for j in range(diffch.shape[1]):
            if(diffch[i, j] < alpha):
                corrected_t[i, j] = dark_t[i, j] * init_t[i, j]
 
    return np.abs(corrected_t)

In [20]:
omega=0.75
alpha=0.4
w=1
get_corrected_transmission(I, A, darkch, brightch, init_t, alpha, omega, w)

array([[0.976, 0.976, 0.976, ..., 0.968, 0.968, 0.96 ],
       [0.976, 0.976, 0.976, ..., 0.968, 0.968, 0.96 ],
       [0.976, 0.976, 0.976, ..., 0.968, 0.968, 0.96 ],
       ...,
       [0.976, 0.976, 0.976, ..., 0.976, 0.976, 0.968],
       [0.976, 0.976, 0.976, ..., 0.976, 0.976, 0.968],
       [0.976, 0.976, 0.976, ..., 0.976, 0.976, 0.968]])